In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [2]:
from dash import Dash, dcc, html, callback, Input, Output
import pandas as pd
import numpy as np
import plotly.express as px
import threading
from google.colab import output, drive

In [6]:
population = pd.read_csv('population.csv')
new_books = pd.read_csv("new-books-per-million.csv")
new_books = new_books[(new_books["Year"]>=1940) & (new_books["Year"]<=1996)]
selected_books = new_books[new_books["Year"]==1996][["Entity", "Book titles per capita (Fink-Jensen 2015)"]]
selected_pops = population[population["Year"]==1996][["Entity", "Population (historical estimates)"]]
data = pd.merge(selected_books, selected_pops, how="inner", on="Entity")

In [7]:
scatter_figure = px.scatter(data_frame=data,
                 x="Book titles per capita (Fink-Jensen 2015)",
                 y="Population (historical estimates)",
                 text="Entity",
                 log_x=True,
                 log_y=True,
                 template="none",
                 title="Country population vs number of new book titles per capita in 1996")
scatter_figure.update_traces(textposition="bottom right")

In [8]:
layout_page_two = html.Div(
    children=[
              dcc.Graph(
                  id='scatter',
                  figure=scatter_figure
                  ),
              dcc.Graph(id='line')
              ],
)

In [9]:
layout_page_one = html.Div(
    children=[
              dcc.Graph(
                  id='some-lines',
                  ),
              dcc.Dropdown(id='country-dropdown', options=new_books["Entity"].unique(), value="Switzerland")
              ],
)

In [10]:
app = Dash()

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),

    dcc.Link('Linechart with dropdown', href='/'),
    html.Br(),
    dcc.Link('Cross filtering in dash', href='/second_page'),

    html.Div(id='page-content')
])

@callback(
    Output('page-content', 'children'),
    Input('url', 'pathname'))
def display_page(pathname):

  if pathname == "/second_page":
    return layout_page_two
  else:
    return layout_page_one

@app.callback(
    Output('line', 'figure'),
    Input('scatter','hoverData')
)
def build_line_chart(country):

  selected_country = country["points"][0]["text"]
  selected_data = new_books[new_books["Entity"]==selected_country]

  fig = px.line(data_frame=selected_data,
                x="Year",
                y="Book titles per capita (Fink-Jensen 2015)",
                template="none",
                title="Book titles per capita in %s for years 1940-1996" % selected_country
                )
  return fig

@app.callback(
    Output('some-lines', 'figure'),
    Input('country-dropdown', 'value')
)
def build_county_line(value):

  present_books = new_books[new_books["Entity"]==value]

  fig = px.line(data_frame=present_books,
              x="Year",
              y="Book titles per capita (Fink-Jensen 2015)",
              color="Entity",
              template="none",
              title="Book titles per capita in different countries for years 1940-1996"
              )
  fig.update_layout(showlegend=False)
  return fig

In [13]:
thread = threading.Thread(target=app.run)
thread.start()
output.serve_kernel_port_as_iframe(port=8050)

<IPython.core.display.Javascript object>